In [1]:
import pandas as pd

### Read data from the wiki page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_tables = pd.read_html(url)

- There are multiple tables in the html. The first table is the list of postal codes. 
- Remove the records with Borough as 'Not assigned'
- Set Neighbourhood with the value of Borough when Neighbourhood is 'Not assigned'

In [3]:
df_toronto = df_tables[0]
df_toronto = df_toronto[df_toronto.Borough!='Not assigned']
df_toronto.loc[df_toronto.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df_toronto[df_toronto.Neighbourhood == 'Not assigned']['Borough']
df_toronto.head(5)

/Users/weikewei/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/weikewei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


- Concat the Neighbourhood values with the same Postcode

In [4]:
df_toronto_agg = df_toronto.groupby('Postcode')['Borough', 'Neighbourhood'].agg(lambda x: max(x) if x.name == 'Borough' else ','.join(x)).reset_index()
df_toronto_agg.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
df_toronto_agg.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


- Show the shape of the result

In [5]:
df_toronto_agg.shape

(103, 3)

- Load geocode from csv

In [6]:
df_geocoders = pd.read_csv('Geospatial_Coordinates.csv')
df_geocoders.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_geocoders.head(5)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


- Join 2 tables on Postal Code

In [7]:
df_postal_codes_geo = pd.merge(df_toronto_agg, df_geocoders, on='PostalCode', how='inner')
df_postal_codes_geo.head(5)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [8]:
df_postal_codes_geo.shape

(103, 5)